<a href="https://colab.research.google.com/github/DhruvaBansal00/ConfidentMT/blob/master/driver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!ls
%cd drive/My Drive/ConfidentMachineTranslation/flores
!ls
!pwd

drive  sample_data
/content/drive/My Drive/ConfidentMachineTranslation/flores
analysis			      FeatureSubsetBinaryClassifers.ipynb
Analysis			      FLORES.ipynb
backward_models			      LanguageAnalysis.ipynb
BoostedBinaryClassifers.ipynb	      language_models
checkpoints			      LM_Thresholding.ipynb
ClassificationDataset		      NCD_Analysis.ipynb
CompareBaselineWithClassifiers.ipynb  NNClassification.ipynb
configs				      noisychannel
data				      NoisyChannel.ipynb
data-bin			      Resources
driver.ipynb			      scripts
Ensembles			      src
Ensembling
/content/drive/My Drive/ConfidentMachineTranslation/flores


In [3]:
pip install fairseq sacrebleu sentencepiece fastBPE sacremoses

     |████████████████████████████████| 307kB 8.9MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 1.1MB 18.0MB/s 
     |████████████████████████████████| 890kB 12.9MB/s 
  Created wheel for fairseq: filename=fairseq-0.9.0-cp36-cp36m-linux_x86_64.whl size=2021170 sha256=f5950774f5523c750bff425d5ebce44aead8235bce2fb6ba44ebb7c37265cb27
  Stored in directory: /root/.cache/pip/wheels/37/3e/1b/0fa30695dcba41e4b0088067fa40f3328d1e8ee78c22cd4766
  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp36-cp36m-linux_x86_64.whl size=481505 sha256=ed7e282b5319bd3b3a36c1726993cf384e567a92a0d04126bf49932cea2a2232
  Stored in directory: /root/.cache/pip/wheels/f3/0c/9c/fc62058b4d473a5602bcd3d3edfece796f123875379ea82d79
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=47c4ce36c4e5c829cd13116be677068dbd8b4eed1247227c36246386cf61f99a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50

In [0]:
##Methods for CLI with fairseq - WIP

def runFairSeqGenerate(dataFolder, sourceLang, targetLang, fwModel, beam, lenpen, dataSet, removeBpe, saveFile):
    !fairseq-generate $dataFolder --source-lang $sourceLang --target-lang $targetLang \
     --path $fwModel --beam $beam --lenpen $lenpen --gen-subset $dataSet --remove-bpe=$removeBpe --sacrebleu > $saveFile


In [4]:
import sys
sys.path.append('/content/drive/My Drive/ConfidentMachineTranslation/flores/src/')
import translationUtils
from translation import Translation

dataFolder = "data-bin/wiki_ne_en_bpe5000/"
fwModel = "Ensembles/April_04.pt"
bwModel = "backward_models/April_17.pt"
lmModel = "language_models/lm/checkpoint_best.pt"
sourceLang = "ne"
targetLang = "en"
dataSet = "test"

translations = translationUtils.getTranslationFromDataset(dataSet, fwModel, bwModel, lmModel, "ne", "en", runFairSeqGenerate, dataFolder)
print(len(translations)) 
for translation in translations:
    print(translation)

  0% 0/14 [00:00<?, ?it/s]/pytorch/aten/src/ATen/native/BinaryOps.cpp:66: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.
2835
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
Non